In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
from datetime import datetime
from meteostat import Point, Hourly

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_error
from lightgbm import LGBMRegressor

from scipy.stats import yeojohnson
from math import exp
import pickle


In [2]:
start = datetime(2014, 11, 1)
end = datetime(2019, 10, 13)

brooklyn = Point(40.646081321775156, -73.95796142054905)
# Getting daily data
data = Hourly(brooklyn, start, end)
data = data.fetch()

In [3]:
df_weather = data.drop(columns= ['dwpt', 'snow', 'wdir', 'wpgt', 'pres', 'tsun'])

df_resampled = df_weather.resample('15T').asfreq().fillna(method='pad')

In [4]:
weather = df_resampled.fillna(0)
weather.isnull().sum()

temp    0
rhum    0
prcp    0
wspd    0
coco    0
dtype: int64

In [5]:
df = pd.read_csv('US Holiday Dates (2004-2021).csv')

In [6]:
# dropping duplicates
df = df.drop_duplicates(subset=['Date'])

In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M')
df['Date'] = pd.to_datetime(df['Date'])

# creating empty DataFrame to accumulate 15-minute intervals for all unique dates
df_intervals = pd.DataFrame()

# creating for loop to convert daily timestamps into 15 minute ones to match other databases
for date in df['Date'].dt.date.unique():
    # Filtering data for the existing dates
    df_day = df[df['Date'].dt.date == date]

    # Creating 15-minute intervals for the existing dates
    intervals = pd.date_range(start=df_day['Date'].min(), end=df_day['Date'].max() + pd.Timedelta(days=1), freq='15T')

    # creating temporary DataFrame within each iteration of the loop
    df_day_intervals = pd.DataFrame({'Date': intervals, 'holiday': 'yes'})

    # Appending the intervals to the final DataFrame
    df_intervals = pd.concat([df_intervals, df_day_intervals], ignore_index=True)


In [8]:
df_intervals.set_index('Date', inplace=True)

In [9]:
holidays = df_intervals

In [10]:
filtered_df =pd.read_csv('flatbush_avenue.csv', index_col=0)

In [11]:
sb_df = filtered_df[filtered_df['Direction'].isin(['sb'])]


In [12]:
nb_df = filtered_df[filtered_df['Direction'].isin(['nb'])]


In [13]:
sb_df.columns = sb_df.columns.str.lower().str.replace(' ', '_')

In [14]:
sb_df = sb_df.rename(columns={'yr': 'year', 'm': 'month', 'd': 'day', 'hh': 'hour', 'mm': 'minute'})
               
sb_df['date'] = pd.to_datetime(sb_df[['year', 'month', 'day', 'hour', 'minute']], format='%Y-%m-%d %H:%M')
sb_df.set_index('date', inplace=True)

In [15]:
refine_df = sb_df[(sb_df['fromst'] == 'brighton_line') & (sb_df['tost'] == 'brighton_line')]


In [16]:
clean_df = refine_df.drop(labels = ['unnamed:_0', 'requestid', 'boro', 'segmentid', 'wktgeom', 'fromst', 'tost', 'street', 'direction'], axis =1)

In [17]:
clean_df.head().sort_index()

,year,month,day,hour,minute,vol
date,,,,,,
2015-05-04 08:00:00,2015,5,4,8,0,176
2015-05-04 08:15:00,2015,5,4,8,15,184
2015-05-04 08:30:00,2015,5,4,8,30,196
2015-05-04 08:45:00,2015,5,4,8,45,180
2015-05-04 09:00:00,2015,5,4,9,0,220


In [18]:
duplicated = clean_df[clean_df.index.duplicated(keep=False)]
duplicated_rows = pd.DataFrame(duplicated)

In [19]:
agg_dict = {'vol': 'sum', 'year': 'first', 'month': 'first', 'day': 'first', 'hour': 'first', 'minute': 'first'}
combined_df = clean_df.groupby(clean_df.index).agg(agg_dict)

duplicated_clean = combined_df[combined_df.index.duplicated(keep=False)]
duplicates = pd.DataFrame(duplicated_clean)


In [20]:
flatbush = combined_df

In [21]:
df_temp = flatbush.merge(weather, left_index=True, right_index=True, how='left')
df_merged = df_temp.merge(holidays, left_index=True, right_index=True, how='left')

In [22]:
df = pd.get_dummies(df_merged, dtype=int)

In [23]:
transformed_data, lmbda = yeojohnson(df.vol)

In [24]:
df['vol'] = transformed_data

In [25]:
df.sort_index()


,vol,year,month,day,hour,minute,temp,rhum,prcp,wspd,coco,holiday_yes
date,,,,,,,,,,,,
2015-05-04 08:00:00,33.853695,2015,5,4,8,0,14.4,57.0,0.0,5.4,0.0,0
2015-05-04 08:15:00,34.788452,2015,5,4,8,15,14.4,57.0,0.0,5.4,0.0,0
2015-05-04 08:30:00,36.159856,2015,5,4,8,30,14.4,57.0,0.0,5.4,0.0,0
2015-05-04 08:45:00,34.323207,2015,5,4,8,45,14.4,57.0,0.0,5.4,0.0,0
2015-05-04 09:00:00,38.803114,2015,5,4,9,0,13.9,59.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-15 22:45:00,44.545653,2018,3,15,22,45,6.1,55.0,0.0,31.7,3.0,0
2018-03-15 23:00:00,42.757065,2018,3,15,23,0,5.6,55.0,0.0,18.4,7.0,0
2018-03-15 23:15:00,46.762648,2018,3,15,23,15,5.6,55.0,0.0,18.4,7.0,0


In [26]:
df['month'] = pd.to_numeric(df['month'])
df['hour'] = pd.to_numeric(df['hour'])

In [27]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(df[col_name]-start_num)/period),
        f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(df[col_name]-start_num)/period)    
             }
    return df.assign(**kwargs).drop(columns=[col_name])

df_feat_hour = generate_cyclical_features(df, 'hour', 24, 0)
df_final = generate_cyclical_features(df_feat_hour, 'month', 12, 1)

In [28]:
df_final_cleaned = df_final.drop(columns=['year', 'day', 'minute'], axis =1)
df_final_cleaned.sort_index().head()

,vol,temp,rhum,prcp,wspd,coco,holiday_yes,sin_hour,cos_hour,sin_month,cos_month
date,,,,,,,,,,,
2015-05-04 08:00:00,33.853695,14.4,57.0,0.0,5.4,0.0,0,0.866025,-0.500000,0.866025,-0.5
2015-05-04 08:15:00,34.788452,14.4,57.0,0.0,5.4,0.0,0,0.866025,-0.500000,0.866025,-0.5
2015-05-04 08:30:00,36.159856,14.4,57.0,0.0,5.4,0.0,0,0.866025,-0.500000,0.866025,-0.5
2015-05-04 08:45:00,34.323207,14.4,57.0,0.0,5.4,0.0,0,0.866025,-0.500000,0.866025,-0.5
2015-05-04 09:00:00,38.803114,13.9,59.0,0.0,0.0,0.0,0,0.707107,-0.707107,0.866025,-0.5


In [29]:
df_final_cleaned = df_final.sort_index()


In [30]:
y = df_final_cleaned[['vol']]
X = df_final_cleaned.drop(columns=['vol'])


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=True, random_state=1)


In [31]:
# Creating a StandardScaler instance
scaler = StandardScaler()

# Fitting the scaler on the training data and transforming it
X_train_scaled = scaler.fit_transform(X_train)

# Transforming the validation and test data using the same scaler
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

y_train_scaled = scaler.fit_transform(y_train)
y_val_scaled = scaler.transform(y_val)
y_test_scaled = scaler.transform(y_test)


In [34]:
param_grid = {
    'num_leaves': [30, 50, 100], #  maximum number of leaves in one tree
    'learning_rate': [0.01, 0.1, 0.2], # step size at each iteration while moving toward a minimum of a loss function
    'n_estimators': [50, 100, 200], # number of trees in the forest
    'colsample_bytree': [0.8, 0.9, 1.0], # regularization technique to prevent overfitting
    'force_col_wise': [True], # provides a speedup for training large datasets
    'verbosity': [0]  # controls the amount of information printed during training
}

lgb_model = LGBMRegressor(objective='regression', metric='l2', boosting_type='gbdt')

# Using GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(lgb_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0)
grid_search.fit(X_train_scaled, y_train_scaled)

print("Best Hyperparameters:", grid_search.best_params_)

# Training the regression model with the best hyperparameters
best_params = grid_search.best_params_
bst = LGBMRegressor(metric='l2', boosting_type='gbdt', **best_params ) # **best_params unpacks the dictionary
bst.fit(X_train_scaled, y_train_scaled, eval_set=[(X_val_scaled, y_val_scaled)])

# Making predictions on the test set
y_pred = bst.predict(X_test_scaled)


Best Hyperparameters: {'colsample_bytree': 0.8, 'force_col_wise': True, 'learning_rate': 0.1, 'n_estimators': 200, 'num_leaves': 100, 'verbosity': 0}


In [46]:
def inverse_transform(scaler, df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df

def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate([predictions.reshape(-1, 1)], axis=0).ravel() # reshaping 'predictions' into 2D array to match 'values'
    df_result = pd.DataFrame(data={"vol": vals, "prediction": preds}, index=df_test.head(len(vals)).index)
    df_result = df_result.sort_index()
    df_result = inverse_transform(scaler, df_result, [["vol", "prediction"]])
    return df_result

df_lgbm = format_predictions(y_pred, y_test_scaled, X_test, scaler)
print(df_lgbm.head())

                           vol  prediction
date                                      
2015-05-04 09:00:00  38.803114   32.765281
2015-05-04 09:45:00  31.807346   31.685193
2015-05-04 11:00:00  33.260598   32.362831
2015-05-04 12:15:00  31.560820   32.627902
2015-05-04 13:15:00  36.497266   34.783751


In [36]:
def invert_yeojhonson(value, lmbda):
  if value>= 0 and lmbda == 0:
    return exp(value) - 1
  elif value >= 0 and lmbda != 0:
    return (value * lmbda + 1) ** (1 / lmbda) - 1
  elif value < 0 and lmbda != 2:
    return 1 - (-(2 - lmbda) * value + 1) ** (1 / (2 - lmbda))
  elif value < 0 and lmbda == 2:
    return 1 - exp(-value)

df_result_lgbm = pd.DataFrame()
df_result_lgbm['vol'] = df_lgbm['vol'].apply(lambda x: invert_yeojhonson(x, lmbda))
df_result_lgbm['prediction'] = df_lgbm['prediction'].apply(lambda x: invert_yeojhonson(x, lmbda))

df_result_lgbm.head()

,vol,prediction
date,,
2015-05-04 09:00:00,220.0,166.869765
2015-05-04 09:45:00,159.0,158.007706
2015-05-04 11:00:00,171.0,163.544494
2015-05-04 12:15:00,157.0,165.731575
2015-05-04 13:15:00,199.0,183.959400


### Evaluating and visualizing results

In [37]:
def calculate_metrics(df):
    return {'mse' : mean_squared_error(df.vol, df.prediction),
            'mae' : mean_absolute_error(df.vol, df.prediction),
            'rmse' : mean_squared_error(df.vol, df.prediction) ** 0.5,
            'r2' : r2_score(df.vol, df.prediction)}


In [38]:
calculate_metrics(df_result_lgbm)

{'mse': 503.1628678698149,
 'mae': 16.388654565388407,
 'rmse': 22.43129215782753,
 'r2': 0.9359075002192138}

In [55]:

with open('lightgbm_model.bin', 'wb') as f_out:
    pickle.dump((bst), f_out)

In [57]:
with open('scaler_model.bin', 'wb') as f_out:
    pickle.dump((scaler, lmbda), f_out)